In [5]:
!pip install timm

In [6]:
import torch
from torchvision import models, transforms
from PIL import Image
#from midas.model_loader import load_model

# Load MiDaS model for depth estimation
#depth_model = load_model("DPT_Large")
depth_model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")
depth_model.eval()

# Load an object detection model (e.g., Faster R-CNN)
detection_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()

# Load and preprocess the image
image_path = "path_to_your_image.jpg"
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
img_tensor = transform(image).unsqueeze(0)




Using cache found in /Users/davidtung/.cache/torch/hub/intel-isl_MiDaS_master


ModuleNotFoundError: No module named 'midas.dpt_depth'

In [ ]:
# Run depth estimation
with torch.no_grad():
    depth_map = depth_model(img_tensor).squeeze().cpu().numpy()

# Run object detection
with torch.no_grad():
    detections = detection_model(img_tensor)


In [ ]:
# Example: Process first detection (assuming it's the object of interest)
if len(detections[0]['boxes']) > 0:
    box = detections[0]['boxes'][0].cpu().numpy()
    x_min, y_min, x_max, y_max = box

    # Estimate depth at the center of the bounding box
    center_x = int((x_min + x_max) / 2)
    center_y = int((y_min + y_max) / 2)
    object_depth = depth_map[center_y, center_x]

    # Approximate 3D location (using estimated depth and camera assumptions)
    f = 800  # Assumed focal length
    c_x, c_y = 320, 240  # Assumed principal point (image center)

    X = (center_x - c_x) * object_depth / f
    Y = (center_y - c_y) * object_depth / f
    Z = object_depth

    print(f"Estimated 3D Location: X={X}, Y={Y}, Z={Z}")
